In [47]:
from pymongo import MongoClient
from dotenv import load_dotenv
import os

load_dotenv()
ATLAS_KEY = os.getenv("ATLAS_KEY")


In [51]:
client = MongoClient(ATLAS_KEY)
db = client.IMDBscrap
collection = db.films

In [49]:
film_le_plus_long = collection.find_one(sort=[("duree", -1)])
film_le_plus_long['titre']

'Gone with the Wind'

In [50]:
titres = collection.distinct("titre", {"_id": {"$in": [doc["_id"] for doc in collection.find().sort([("score", -1)]).limit(5)]}})


titres


['The Dark Knight',
 'The Godfather',
 'The Godfather Part II',
 'The Lord of the Rings: The Return of the King',
 'The Shawshank Redemption']